# ML com Árvores de Decisão

Nesse notebook vamos criar nossos primeiros modelos estatísticos usando **árvores de decisão**, aprender sobre diferentes estratégias de aprendizado e tipos de modelos, como classificadores e regressores.

In [ ]:
import matplotlib.pyplot as plt
import seaborn
import pandas as pd
import numpy as np
import utils

%matplotlib inline
%load_ext autoreload
%autoreload 2

plt.rcParams['figure.figsize'] = (8.0, 5.0)

## Por que eu preciso de outro método?

Vamos primeiro revisitar um exemplo onde nossos métodos lineares funcionam muito bem!

In [ ]:
heights_x, heights_y = utils.load_heights()

In [ ]:
plt.plot(heights_x, heights_y, '.')

Vamos usar Regressão Linear nos nossos dados pra ver se um modelo consegue descrever o comportamento que estamos vendo.

Vamos plotar nosso modelo junto dos nossos pontos:

Podemos até dar um palpite usando nosso modelo pra diferentes alturas (69 inches?)

Parece bom :)

## Prevendo a maré

Agora vamos pra um problema diferente. Queremos saber qual a altura da maré em uma praia da Bahia dado o horário do dia:

In [ ]:
x, y = utils.load_tides()

In [ ]:
plt.figure(figsize(8,4))
plt.xlim(0, 24)
plt.xlabel("Hora do dia")
plt.ylabel("Metros")

plt.scatter(x, y, label="data")

Vamos tentar usar regressão linear novamente e plotar nosso modelo:

EITA!

O modelo simplesmente não é capaz de descrever o comportamento que estamos tentando prever.

Agora vamos usar uma **árvore de decisão** (para regressão) e comparar os resultados:

## Frutas

In [ ]:
fruits_x = [[150, "rugosa", "laranja"],
            [170, "rugosa", "verde"],
            [140, "lisa", "vermelha"],
            [130, "lisa", "vermelha"],
            [136, "lisa", "verde"]]

fruits_y = ["laranja", "laranja", u"maça", u"maça", u"maça"]

`fruits_x` são as nossas features (as vezes chamadas de atributos)

`fruits_y` é o nosso target, ou seja, o que queremos inferir/prever a partir de X (também chamado de label)

Antes de treinar nosso modelo, temos que fazer todas **nossas features serem numéricas**, pois é o formato esperado pelos algoritmos do `sklearn`:

In [ ]:
fruits_x = [[150, 0, 0],
            [170, 0, 1],
            [140, 1, 2],
            [130, 1, 2],
            [136, 1, 1]]

Nosso modelo já aprendeu as melhores regras para distinguir maças e laranjas. Agora podemos classificar novos objetos que não conhecíamos antes:

Nosso objeto `dt` é o que chamamos de um **modelo**. Ele foi treinado usando um algoritmo de árvores de decisão. Poderíamos ter usado outro algoritmo, que apesar de funcionar de forma diferente, também aprenderia regras para prever novos dados:

## Usando Árvores de Decisão

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, export_graphviz
import graphviz

### Dataset Iris

Esse dataset contém medidas da largura e comprimento da pétula e da sépala de 150 amostras de flores, assim como a espécie de cada uma (versicolor, setosa, virginica).

![oranges_apples_ilustracao.png](images/iris.png)

Vamos carregar o dataset e treinar uma árvore de decisão nos nossos dados.

In [ ]:
x, y, species = utils.load_flowers()

Vamos definir um conjunto `x_test` para testarmos nosso algoritmo. Os primeiros? Aleatórios?

Agora como podemos avaliar se nosso modelo é preciso? Vamos definir uma métrica simples chamada acurácia, que basicamente é a porcentagem de valores para os quais previmos a classe certa:

In [ ]:
accuracy(y_test, y_pred)

Porém esse número parece confiável? Basicamente ele diz que nosso classificador é perfeito e sempre acerta a espécie da flor. Vamos dar uma olhada na nossa árvore:

In [ ]:
dot_data = export_graphviz(dt,
                           out_file=None, 
                           feature_names=x.columns,
                           class_names=species,  
                           filled=True, rounded=True)
graph = graphviz.Source(dot_data)
graph

Podemos observar que, a medida que aprofundamos na árvore, alguns nós muito específicos aparecem. Muitos nós separam somente um exemplo da nossa base. É difícil de acreditar que, dentre 150 amostras, uma regra que separa somente 1 exemplo seja de fato algo logicamente (nesse caso, biologicamente) válido.

Assim, para termos uma estimativa melhor do desempenho do nosso algoritmo, vamos reservar uma parte dos dados que não serão usados no treino, mas somente na avaliação (ou como chamamos, no teste).

## Exercício: Cancer de mama

Nesse exemplo vocês vão treinar uma árvore de decisão para classificar se um tumor é benigno ou maligno. O que deve ser feito:

* Liste os atributos/features do problema.
* Calcule a média, desvio padrão, minimo e máximo de cada atributo (dica, pandas tem um método muito conveniente para isso).
* Observe se o problema é balanceado (o que é isso mesmo? e por que isso é relevante?)
* Separe um conjunto de teste e um de treino (quanto pra cada?)
* Escolha uma métrica de avaliação e avalie o classificador no conjunto de treino e teste.
* Plote a árvore treinada.

In [ ]:
x, y, target_names = utils.load_dataset('cancer')

In [ ]:
dot_data = export_graphviz(dt, 
                           out_file=None,
                           feature_names=x.columns,
                           class_names=target_names,
                           filled=True, rounded=True, special_characters=True)
graphviz.Source(dot_data)

## Overfitting e Underfitting

### Exercício

Usando os dados acima de cancer de mama, você deve:

* Treinar vários modelos variando o valor do parâmetro `max_depth` entre 1 e 10
* Plotar um gráfico com duas curvas, `max_depth vs acurácia` no conjunto de treino, `max_depth vs acurácia` no conjunto de teste.

Dicas: para os plots, use o método pronto `plot_accs(values, accs_train, accs_test, param_name)`.

In [ ]:
def plot_accs(values, accs_train, accs_test):
    plt.plot(values, accs_train, label='train')
    plt.plot(values, accs_test, label='test')
    plt.ylabel('Accuracy')
    plt.xlabel('Max Depth')
    plt.legend()


depths = list(range(1, 10))

for depth in depths:
    dt = DecisionTreeClassifier(random_state=1, max_depth=depth)
#     ...

    
# plot_accs(values, accs_train, accs_test)

Podemos perceber que o desempenho no conjunto de teste cai a medida que o modelo fica mais complexo tentando se adequar ao conjunto de treino (valores maiores de `max_depth`)

## Melhores parâmetros

In [ ]:
dt = DecisionTreeClassifier(max_depth=?, random_state=87)
dt.fit(xtrain, ytrain)

print('Acurácia no treino:', accuracy(ytrain, dt.predict(xtrain)))
print('Acurácia no teste:', accuracy(ytest, dt.predict(xtest)))

In [ ]:
dot_data = export_graphviz(dt, 
                           out_file=None, 
                           feature_names=xtrain.columns,
                           class_names=target_names,
                           filled=True, rounded=True, 
                           special_characters=True)
graphviz.Source(dot_data)

## Análise e Exploração

In [ ]:
x, y = utils.load_dataset('fraud')

In [ ]:
dt = DecisionTreeClassifier(max_depth=4, random_state=42)
dt.fit(x, y)

dot_data = export_graphviz(dt, out_file=None,
                           feature_names=x.columns,
                           class_names=['legit', 'fraud'],  
                           filled=True, 
                           rounded=True,  
                           special_characters=True)
graphviz.Source(dot_data)

# Kaggle Competition

![Comp](images/taxi-competition.png)

Vamos construir um modelo pra tentar prever quanto tempo uma viagem de taxi em NY vai levar. Depois vamos submeter nossa solução para o Kaggle.

Primeira coisa que devemos fazer é baixar os arquivos, descompactar e carregá-los no jupyter:

Vamos olhar rapidamente algumas características dos nossos dados:
- Quantos exemplos temos?
- Quantas features?
- Alguma variável não-numérica?

Vamos também ver se temos valores faltantes (null) na nossa base. A maioria dos algoritmos não saberá lidar com esses valores.

Separar nossas features do nosso target e reservar um conjunto de teste para avaliarmos nossos modelos:

Vamos ver a distruição do nosso target:

Antes de treinarmos nosso primeiro modelo, temos que tratar nossas variáveis não numéricas (vamos só ignorar por enquanto):

Agora vamos fazer nosso split para avaliar nossos modelos:

E rodar as predição e avaliar tanto no treino quanto no teste:

Antes de submeter, vamos usar a métrica oficial da competição para termos uma ideia de onde podemos nos colocar:

Bora criar uma submissão?

# FIM